# Regressió logística aplicada a iris

**Lectura del corpus i partició:**

In [ ]:
# Importar la biblioteca NumPy per a càlculs numèrics
import numpy as np

# Importar la funció per carregar el conjunt de dades Iris des de scikit-learn
from sklearn.datasets import load_iris

# Importar la funció per dividir dades en conjunts d'entrenament i prova
from sklearn.model_selection import train_test_split

# Carregar el conjunt de dades Iris
iris = load_iris()

# Extreure les característiques i convertir-les al tipus de dada float16 per optimitzar l'ús de memòria
X = iris.data.astype(np.float16)

# Extreure les etiquetes i convertir-les al tipus de dada enter sense signe (uint)
y = iris.target.astype(np.uint)

# Dividir les dades en conjunts d'entrenament i prova, amb un 20% de les dades com a prova
# 'shuffle=True' assegura que les dades es barregen abans de dividir-se
# 'random_state=23' assegura que la divisió siga reproductible en utilitzar una llavor específica
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


**LogisticRegression:** $\;$ implementació de regressió logística en sklearn

In [ ]:
# Importar la classe LogisticRegression de scikit-learn per realitzar regressió logística
from sklearn.linear_model import LogisticRegression

# Importar la funció accuracy_score per calcular la precisió de les prediccions
from sklearn.metrics import accuracy_score

# Crear una instància del classificador de regressió logística amb una llavor aleatòria per a la reproductibilitat
clf = LogisticRegression(random_state=23).fit(X_train, y_train)

# Predir les etiquetes per al conjunt de dades de prova
y_test_pred = clf.predict(X_test)

# Calcular l'error en el conjunt de prova com 1 menys la precisió
err_test = 1 - accuracy_score(y_test, y_test_pred)

# Imprimir l'error de prova formatat com un percentatge amb un decimal
print(f"Error de prova: {err_test:.1%}")


**Warnings:** $\;$ sklearn és un poc "llandós" amb el warnings; ignorarem els avisos sobre convergència

In [ ]:
# Importar el mòdul warnings per controlar les advertències
import warnings

# Importar ConvergenceWarning de sklearn.exceptions per capturar advertències específiques de convergència
from sklearn.exceptions import ConvergenceWarning

# Configurar el mòdul warnings per ignorar totes les advertències de la categoria ConvergenceWarning que provenen de la biblioteca sklearn
warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")


**Solvers:** $\;$ el paràmetre `solver` de LogisticRegression permet triar entre diferents solvers (algorismes d'optimització)

In [ ]:
# Definir una llista de solvers per provar amb el model de regressió logística
for solver in ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']:
    # Crear i entrenar el model de regressió logística amb cada solver
    # 'max_iter=10000' estableix un nombre alt d'iteracions per permetre la convergència
    clf = LogisticRegression(random_state=23, solver=solver, max_iter=10000).fit(X_train, y_train)

    # Predir les etiquetes per al conjunt de prova i calcular l'error
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))

    # Imprimir l'error de prova per cada solver
    print(f"Error de prova després d'entrenar amb el solver {solver!s}: {err_test:.1%}")


**Tolerància:** $\;$ el paràmetre `tol` estableix un llindar de tolerància per a acabar l'entrenament (1e4 per omissió)

In [ ]:
# Iterar sobre una llista de diferents valors de tolerància per al criteri de parada
for tol in (1e-4, 1e-2, 1, 1e2, 1e4):
    # Crear i entrenar el model de regressió logística amb cada valor de tolerància
    # 'tol' defineix el criteri de parada per a l'optimització: el procés s'atura quan l'error és menor que 'tol'
    clf = LogisticRegression(tol=tol, random_state=23, max_iter=10000).fit(X_train, y_train)

    # Predir les etiquetes per al conjunt de prova i calcular l'error
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))

    # Imprimir l'error de prova per cada valor de tolerància
    print(f"Error de prova amb tolerància {tol}: {err_test:.1%}")

# Els valors de 'tol' van des de molt petits (1e-4) fins a molt grans (1e4), la qual cosa permet avaluar com la precisió del model varia amb diferents criteris de parada en l'optimització.


**Regularització:** $\;$ el paràmetre `C` (positiu, $1.0$ per omissió) des-regularitza el criteri d'entrenament
* **Possibilitat de subajust:** $\;$ amb un valor pròxim a zero (màxima regularització)
* **Possibilitat de sobreajust:** $\;$ amb un valor positiu molt alt (mínima regularització)

In [ ]:
for C in (1e-2, 1e-1, 1, 1e1, 1e2):
    clf = LogisticRegression(C=C, random_state=23, max_iter=10000).fit(X_train, y_train)
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))
    print(f"Error de test amb C {C:g}: {err_test:.1%}")

**Early stopping:** $\;$ estaviem càlcul i evitem sobre-entrenament ("regularitzem") acabant prompte (en poques iteracions)

In [ ]:
# Aquest codi avalua com la limitació en el nombre màxim d'iteracions afecta el rendiment del model.
# Valors baixos de 'max_iter' poden portar a una convergència prematura, possiblement resultant en un model subòptim.

# Iterar sobre una llista de diferents valors per al nombre màxim d'iteracions
for max_iter in (10, 20, 50, 100):
    # Crear i entrenar el model de regressió logística amb cada valor de max_iter
    # 'max_iter' defineix el nombre màxim d'iteracions del procés d'optimització
    clf = LogisticRegression(random_state=23, max_iter=max_iter).fit(X_train, y_train)

    # Predir les etiquetes per al conjunt de prova i calcular l'error
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))

    # Imprimir l'error de prova per cada valor de max_iter
    print(f"Error de prova amb max_iter {max_iter}: {err_test:.1%}")


**Ho posem tot junt**  
---

Mostrar els resultats d'error de test (**err_test**) per a combinacions de diferents valors de:

1. `max_iter` (provar valors 10, 50, 100, 200, 500, 800, 1000)  
2. `solver` (utilitzar **sag**, **saga**, **lbfgs** i **newton-cg**, que són els solvers per a problemes multiclasse)  
3. `C` (regularització: 0.01, 0.1, 1.0, 10.0, 100)  
4. `tol` (tolerància: provar valors 10e-4, 10e-2, 1.0, 10e2, 10e4)  


In [ ]:
# Definir els paràmetres a provar
max_iters = [10, 50, 100, 200, 500, 800, 1000]
solvers = ['sag', 'saga', 'lbfgs', 'newton-cg']
Cs = [0.01, 0.1, 1.0, 10.0, 100]
tols = [1e-4, 1e-2, 1, 1e2, 1e4]

print('   solver     tol       C max_iter  ete')
print('--------- ------- ------- -------- -----')

# Iterar sobre las combinaciones de max_iter, solver, C y tol
for max_iter in max_iters:
    for solver in solvers:
        for C in Cs:
            for tol in tols:
                # Crear y entrenar el modelo
                clf = LogisticRegression(max_iter=max_iter, solver=solver, C=C, tol=tol, random_state=23).fit(X_train, y_train)
                # Calcular el error de prueba
                err_test = 1 - accuracy_score(y_test, clf.predict(X_test))
                # Imprimir los resultados
                print(f'{solver:>9} {tol:.1e} {C:.1e} {max_iter:8d} {ete:5.1%}')

**Pregunta**:  
Indica quina creus que és la millor combinació d'iteracions, solver, C i tolerància.

**Modificació:**  
Modifica el codi anterior per calcular també l'error en el conjunt d'entrenament, tenint en compte **X_train** i **y_train**.

In [ ]:
#inserta el codi